In [98]:
from math import floor
from PIL import Image
from scipy import ndimage
import numpy as np
import os
import os.path
import time
import random
import cv2


class Dither():
    def __init__(self, path, scale=1, amount=0.001, save=True):
        self.path = self.get_path(path)
        self.func = self.get_func()
        self.scale = self.get_scale(scale)
        self.amount = self.get_amount(amount)
        self.save = self.get_save(save)
        self.func(self.path, self.scale, self.amount, self.save)
        
    def get_save(self, save):
        return save
    
    def get_scale(self, scale):
        return scale
    
    def get_amount(self, amount):
        return amount
        
    def get_path(self, path):
        """Get whole path of an image

        If path does not start with '/', then try to open image from pwd
        If path starts with '/', then open image of given path
        """
        if path.startswith('/') and not path.startswith('~/'):
            return os.getcwd() + '/' + path
        else:
            return path

    def get_func(self):
        "Get dithering function to run"
        return self.FloydSteinberg
              
    def add_noise(self, image, amount):
        #Gets image array input and adds black speckles
        
        num_pepper = np.ceil(amount * image.size)
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        image[tuple(coords)] = 0

        return image
    
    def pre_process(self, image, amount):
        
        #Convert image to scipy array
        img = np.array(image)

        # Edge detection
        img = cv2.Canny(img, 150, 300)
        
        #Dilat edges to make them bigger
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        img = cv2.dilate(img, kernel, iterations=1)
        
        # Inverse image
        img = cv2.bitwise_not(img)
        
        # Remove points
        img.tolist()
        xlim, ylim = len(img[0]), len(img)
        
        for y in range(0, ylim):
            for x in range(0, xlim):
                #Only alter black pixels
                if not img[y, x]:
                    if random.uniform(0, 1) >= 0.05:
                        img[y, x] = 255
        img = np.array(img)

        
        # Add Noise
        #img = self.add_noise(img, amount)
        
        #Convert to PIL
        out = Image.fromarray(img)
        
        return out
    
    def save_image(self, image, amount, scale):
        """Saves (pillow) image to 'output' folder"""
        
        save_path = r"C:\\Users\\micha\\OneDrive - TU Eindhoven\\00. Studie Relevant\\2IMV10\\Python\\Outputs\\"
        folderName = save_path + "output" + time.strftime("%d%b%Y--%H-%M-%S")
        os.makedirs(folderName)
        
        #Save image
        image.save(folderName + "\\image.png")
                
        #Save settings
        settingsName = os.path.join(folderName, "settings.txt")         
        file = open(settingsName, "w")
        
        file.write("amount:{}\nscale:{}".format(amount, scale))
        
        #Save pixels
        completeName = os.path.join(folderName, "pixels.txt")         
        file = open(completeName, "w")
        
        file.write("[")
        
        image = np.array(image)
        image.tolist()
        xlim, ylim = len(image[0]), len(image)
        
        for y in range(0, ylim):
            for x in range(0, xlim):
                
                #Only output black pixels
                if not image[y, x]:
                    file.write("[{}, {}],\n".format(x, y))
                    
        file.write("];")        

    def FloydSteinberg(self, image_file, scale, amount, save):
        """Perform Floyd-Steinberg dithering"""
        
        #Open image
        
        img = Image.open(image_file)
        
        #Resizing image
        
        width, height = img.size
        size = (floor(width/scale), floor(height/scale))
        img = img.resize(size)
        
        #Pre-processing
        
        # Convert to b&w
        img = img.convert('L')
        img = self.pre_process(img, amount)

        #Floyd-Steinberg dithering
        img = img.convert('1')

        #Show & save pixels from image
        img.show()
        if save:
            self.save_image(img, amount, scale)

def main():
    #Call dithering algorithm
    Dither(image_path, image_scale, image_amount)



In [99]:
Dither(r"C:\Users\micha\OneDrive - TU Eindhoven\00. Studie Relevant\2IMV10\Python\img\abe.jpg", scale=1, amount=0.0001, save=True)